# Gaussian processes

In this tutorial, we explain how to obtain the functionality of other packages such as GaussianProcesses.jl for Gaussian processes in Julia with GeoStats.jl. Gaussian processes (the method) and Simple Kriging are essentially [two names for the same concept](https://en.wikipedia.org/wiki/Kriging). The derivation of Kriging estimators, however; does **not** require distributional assumptions.

[Matheron](https://en.wikipedia.org/wiki/Georges_Matheron) and other important geostatisticians have generalized Gaussian processes to random fields with locally-varying mean and for situations where the mean is unknown. GeoStats.jl includes Gaussian processes as a special case as well as other more practical Kriging variants.

In machine learning, Gaussian processes are usually explained in 1D, and we will do the same here.

Before we proceed, please install the following packages:

In [1]:
for pkg in ["GeoStats", "Plots", "GR", "Interact"]
    Pkg.add(pkg)
end

# make sure this tutorial is reproducible
srand(2000);

INFO: Package GeoStats is already installed
INFO: Package Plots is already installed
INFO: Package GR is already installed
INFO: Package Interact is already installed


## The data

Let's start by drawing a set of points in the real line, each with a known value:

In [2]:
using Plots
gr(size=(1000,400), xlim=(0,1), ylim=(0,1))

x = collect(linspace(0.1,.9,10))
z = rand(10)

scatter(x, z, label="data", size=(1000,400))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data

We will skip an important step known in geostatistics as *variogram modeling*, and will proceed to the estimation step directly in order to avoid having too many concepts in this tutorial. These concepts are likely new to practioners of machine learning and can be very useful in practice. For more information on variogram modeling, please check the other tutorials in this folder.

## Simple Kriging

In simple Kriging (or Gaussian process), we need to specify a mean and a covariance-type function in order to interpolate the stochastic process at unseen locations. We choose a covariance (or variogram) model in GeoStats.jl and interact with the parameters:

In [3]:
using GeoStats
using Interact

# discretize the line segment for plotting
n = 200
xs = linspace(0,1,n)

@manipulate for s=linspace(.0,.1), r=linspace(.1,.2)
    # some given mean
    process_mean = .5
    
    # spatial structure
    γ = GaussianVariogram(sill=s, range=r)
    
    # create Gaussian process
    simkrig = SimpleKriging(x', z, γ, process_mean)
    
    # estimate mean and variance
    μ = zeros(n)
    σ² = zeros(n)
    for (i,loc) in enumerate(xs)
        μ[i], σ²[i] = estimate(simkrig, [loc])
    end
    
    σ = sqrt.(σ²)
    
    scatter(x, z, label="data")
    plot!(xs, μ, ribbon=σ, fillalpha=.5, label="Gaussian process")
end

Interact.Options{:SelectionSlider,Any}(1: "input" = 0.04897959183673469 Any , "s", 0.04897959183673469, "0.04897959183673469", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict{Any,Any}(Pair{Any,Any}(0.0102041, "0.01020408163265306"),Pair{Any,Any}(0.0183673, "0.018367346938775512"),Pair{Any,Any}(0.0265306, "0.026530612244897958"),Pair{Any,Any}(0.0877551, "0.08775510204081632"),Pair{Any,Any}(0.0163265, "0.0163265306122449"),Pair{Any,Any}(0.0693878, "0.06938775510204082"),Pair{Any,Any}(0.0857143, "0.08571428571428572"),Pair{Any,Any}(0.0306122, "0.030612244897959183"),Pair{Any,Any}(0.0387755, "0.03877551020408163"),Pair{Any,Any}(0.0367347, "0.036734693877551024")…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Any}(3: "input-2" = 0.1489795918367347 Any , "r", 0.1489795918367347, "0.1489795918367347", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.1"=>0.1,"0.10204081632653061"=>0.102041,"0.10408163265306122"=>0.104082,"0.10612244897959183"=>0.106122,"0.10816326530612246"=>0.108163,"0.11020408163265306"=>0.110204,"0.11224489795918367"=>0.112245,"0.11428571428571428"=>0.114286,"0.11632653061224489"=>0.116327,"0.11836734693877551"=>0.118367…), Dict{Any,Any}(Pair{Any,Any}(0.146939, "0.1469387755102041"),Pair{Any,Any}(0.112245, "0.11224489795918367"),Pair{Any,Any}(0.128571, "0.12857142857142856"),Pair{Any,Any}(0.12449, "0.12448979591836734"),Pair{Any,Any}(0.189796, "0.18979591836734694"),Pair{Any,Any}(0.187755, "0.18775510204081633"),Pair{Any,Any}(0.122449, "0.12244897959183673"),Pair{Any,Any}(0.2, "0.2"),Pair{Any,Any}(0.134694, "0.1346938775510204"),Pair{Any,Any}(0.169388, "0.16938775510204082")…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data 
 
 
 
 Gaussian process

## Ordinary Kriging

An immediate generalization of Simple Kriging is Ordinary Kriging. In this case, we still have the assumption that the mean of the field is constant, but this time we treat it as an unknown. Below is the estimation with Ordinary Kriging where we only need to specify the covariance-type function:

In [4]:
using GeoStats
using Interact

# discretize the line segment for plotting
n = 200
xs = linspace(0,1,n)

@manipulate for s=linspace(.0,.1), r=linspace(.1,.2)
    # spatial structure
    γ = GaussianVariogram(sill=s, range=r)
    
    # create Gaussian process
    ordkrig = OrdinaryKriging(x', z, γ)
    
    # estimate mean and variance
    μ = zeros(n)
    σ² = zeros(n)
    for (i,loc) in enumerate(xs)
        μ[i], σ²[i] = estimate(ordkrig, [loc])
    end
    
    σ = sqrt.(σ²)
    
    scatter(x, z, label="data")
    plot!(xs, μ, ribbon=σ, fillalpha=.5, label="Gaussian process")
end

Interact.Options{:SelectionSlider,Any}(7: "input-3" = 0.04897959183673469 Any , "s", 0.04897959183673469, "0.04897959183673469", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.0"=>0.0,"0.0020408163265306124"=>0.00204082,"0.004081632653061225"=>0.00408163,"0.006122448979591836"=>0.00612245,"0.00816326530612245"=>0.00816327,"0.01020408163265306"=>0.0102041,"0.012244897959183673"=>0.0122449,"0.014285714285714285"=>0.0142857,"0.0163265306122449"=>0.0163265,"0.018367346938775512"=>0.0183673…), Dict{Any,Any}(Pair{Any,Any}(0.0102041, "0.01020408163265306"),Pair{Any,Any}(0.0183673, "0.018367346938775512"),Pair{Any,Any}(0.0265306, "0.026530612244897958"),Pair{Any,Any}(0.0877551, "0.08775510204081632"),Pair{Any,Any}(0.0163265, "0.0163265306122449"),Pair{Any,Any}(0.0693878, "0.06938775510204082"),Pair{Any,Any}(0.0857143, "0.08571428571428572"),Pair{Any,Any}(0.0306122, "0.030612244897959183"),Pair{Any,Any}(0.0387755, "0.03877551020408163"),Pair{Any,Any}(0.0367347, "0.036734693877551024")…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Any}(9: "input-4" = 0.1489795918367347 Any , "r", 0.1489795918367347, "0.1489795918367347", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("0.1"=>0.1,"0.10204081632653061"=>0.102041,"0.10408163265306122"=>0.104082,"0.10612244897959183"=>0.106122,"0.10816326530612246"=>0.108163,"0.11020408163265306"=>0.110204,"0.11224489795918367"=>0.112245,"0.11428571428571428"=>0.114286,"0.11632653061224489"=>0.116327,"0.11836734693877551"=>0.118367…), Dict{Any,Any}(Pair{Any,Any}(0.146939, "0.1469387755102041"),Pair{Any,Any}(0.112245, "0.11224489795918367"),Pair{Any,Any}(0.128571, "0.12857142857142856"),Pair{Any,Any}(0.12449, "0.12448979591836734"),Pair{Any,Any}(0.189796, "0.18979591836734694"),Pair{Any,Any}(0.187755, "0.18775510204081633"),Pair{Any,Any}(0.122449, "0.12244897959183673"),Pair{Any,Any}(0.2, "0.2"),Pair{Any,Any}(0.134694, "0.1346938775510204"),Pair{Any,Any}(0.169388, "0.16938775510204082")…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data 
 
 
 
 Gaussian process

# Other generalizations

- We can also have random fields with non-constant mean. In Universal Kriging, the mean is a polynomial on the coordinates of the problem up to a certain degree specified by the user. In External Drift Kriging, the mean can be any external variable (a.k.a. external drift).

- All methods in GeoStats.jl accept general distance functions. We can for example utilze the Haversine distance to compute covariances between latitude/longitude coordinates directly.

- Other methods for co-estimation and co-simulation will be available in future releases.